In [ ]:
# Initial Python environment setup...
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to

print(os.listdir('../input'))

In [ ]:
train_data =  pd.read_csv('../input/train.csv', nrows = 10_000_000)
train_data.dtypes

In [ ]:
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_data)

In [ ]:
print(train_data.isnull().sum())

In [ ]:
train_data = train_data.dropna(how = 'any', axis = 'rows')

In [ ]:
plot = train_data.iloc[:2000].plot.scatter('abs_diff_longitude', 'abs_diff_latitude')

In [ ]:
train_data = train_data[(train_data.abs_diff_longitude < 5.0) & (train_data.abs_diff_latitude < 5.0)]

In [ ]:
test_data = pd.read_csv('../input/test.csv')
test_data.dtypes

In [ ]:
add_travel_vector_features(test_data)

In [ ]:
test_data.dtypes

In [ ]:
train_data.dtypes

In [ ]:
train_data.head()

In [ ]:
ls1=list(train_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-7:]
train_data['pickup_time']=ls1

ls1=list(test_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-7:]
test_data['pickup_time']=ls1


In [ ]:

ls1=list(train_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=pd.Timestamp(ls1[i][:-4:]).weekday()
train_data['weekday']=ls1

ls1=list(test_data['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i]=pd.Timestamp(ls1[i][:-4:]).weekday()
test_data['weekday']=ls1

In [ ]:
train_data.drop('pickup_datetime',inplace=True,axis=1)
test_data.drop('pickup_datetime',inplace=True,axis=1)

In [ ]:
train_data.head(10)

In [ ]:
train_data['weekday'].replace(to_replace=[i for i in range(0,7)],value=['Monday','Tuesday','Wednesday','Thrusday','Friday','Saturday','Sunday'],inplace=True)
test_data['weekday'].replace(to_replace=[i for i in range(0,7)],value=['Monday','Tuesday','Wednesday','Thrusday','Friday','Saturday','Sunday'],inplace=True)

In [ ]:
train_one_shot=pd.get_dummies(train_data['weekday'])
test_one_shot=pd.get_dummies(test_data['weekday'])
test_data=pd.concat([test_data,test_one_shot],axis=1)
train_data=pd.concat([train_data,train_one_shot],axis=1)

In [ ]:
train_data.drop('weekday',inplace=True,axis=1)
test_data.drop('weekday',inplace=True,axis=1)

In [ ]:
ls1=list(train_data['pickup_time'])
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(z[0])*100+int(z[1])
train_data['pickup_time']=ls1

ls1=list(test_data['pickup_time'])
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(z[0])*100+int(z[1])
test_data['pickup_time']=ls1

In [ ]:
R=6372.0
lat1=np.asarray(np.radians(train_data['pickup_latitude']))
lon1=np.asarray(np.radians(train_data['pickup_longitude']))
lat2=np.asarray(np.radians(train_data['dropoff_latitude']))
lon2=np.asarray(np.radians(train_data['dropoff_latitude']))

dlon=lon2-lon1
dlat=lat2-lat1
ls1=[]
a=np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
distance=R*c

train_data['Distance']=np.asarray(distance)*0.621


lat1=np.asarray(np.radians(test_data['pickup_latitude']))
lon1=np.asarray(np.radians(test_data['pickup_longitude']))
lat2=np.asarray(np.radians(test_data['dropoff_latitude']))
lon2=np.asarray(np.radians(test_data['dropoff_latitude']))

dlon=lon2-lon1
dlat=lat2-lat1
ls1=[]
a=np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
distance=R*c

test_data['Distance']=np.asarray(distance)*0.621

In [ ]:
R=6372.0
lat1=np.asarray(np.radians(train_data['pickup_latitude']))
lon1=np.asarray(np.radians(train_data['pickup_longitude']))
lat2=np.asarray(np.radians(train_data['dropoff_latitude']))
lon2=np.asarray(np.radians(train_data['dropoff_latitude']))

lat3=np.zeros(len(train_data))+np.radians(40.641)
lon3=np.zeros(len(train_data))+np.radians(-73.778)
dlon_pickup=lon3-lon1
dlat_pickup=lat3-lat1
dlon_dropoff=lon3-lon2
dlat_dropoff=lat3-lat2
a1=np.sin(dlat_pickup/2)**2+np.cos(lat1)*np.cos(lat3)*np.sin(dlon_pickup/2)**2
c1=2*np.arctan2(np.sqrt(a1),np.sqrt(1-a1))
distance1=R*c1
train_data['pickup_distance_a']=np.asarray(distance1)*0.621

a2=np.sin(dlat_dropoff/2)**2+np.cos(lat2)*np.cos(lat3)*np.sin(dlon_dropoff/2)**2
c2=2*np.arctan2(np.sqrt(a2),np.sqrt(1-a2))
distance2=R*c2

train_data['dropoff_distance_a']=np.asarray(distance2)*0.621

lat1=np.asarray(np.radians(test_data['pickup_latitude']))
lon1=np.asarray(np.radians(test_data['pickup_longitude']))
lat2=np.asarray(np.radians(test_data['dropoff_latitude']))
lon2=np.asarray(np.radians(test_data['dropoff_latitude']))

lat3=np.zeros(len(test_data))+np.radians(40.641)
lon3=np.zeros(len(test_data))+np.radians(-73.778)
dlon_pickup=lon3-lon1
dlat_pickup=lat3-lat1
dlon_dropoff=lon3-lon2
dlat_dropoff=lat3-lat2
a1=np.sin(dlat_pickup/2)**2+np.cos(lat1)*np.cos(lat3)*np.sin(dlon_pickup/2)**2
c1=2*np.arctan2(np.sqrt(a1),np.sqrt(1-a1))
distance1=R*c1
test_data['pickup_distance_a']=np.asarray(distance1)*0.621

a2=np.sin(dlat_dropoff/2)**2+np.cos(lat2)*np.cos(lat3)*np.sin(dlon_dropoff/2)**2
c2=2*np.arctan2(np.sqrt(a2),np.sqrt(1-a2))
distance2=R*c2

test_data['dropoff_distance_a']=np.asarray(distance2)*0.621


In [ ]:
train_data['Distance']=np.round(train_data['Distance'],2)
train_data['pickup_distance_a']=np.round(train_data['pickup_distance_a'],2)
train_data['dropoff_distance_a']=np.round(train_data['dropoff_distance_a'],2)

test_data['Distance']=np.round(test_data['Distance'],2)
test_data['pickup_distance_a']=np.round(test_data['pickup_distance_a'],2)
test_data['dropoff_distance_a']=np.round(test_data['dropoff_distance_a'],2)

In [ ]:
train_data.drop(['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude'],inplace=True,axis=1)
test_data.drop(['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude'],inplace=True,axis=1)

In [ ]:
train_data['abs_diff_longitude']=train_data['abs_diff_longitude']-np.mean(train_data['abs_diff_longitude'])
train_data['abs_diff_longitude']=train_data['abs_diff_longitude']/np.var(train_data['abs_diff_longitude'])

train_data['abs_diff_latitude']=train_data['abs_diff_latitude']-np.mean(train_data['abs_diff_latitude'])
train_data['abs_diff_latitude']=train_data['abs_diff_latitude']/np.var(train_data['abs_diff_latitude'])

test_data['abs_diff_longitude']=test_data['abs_diff_longitude']-np.mean(test_data['abs_diff_longitude'])
test_data['abs_diff_longitude']=test_data['abs_diff_longitude']/np.var(test_data['abs_diff_longitude'])

test_data['abs_diff_latitude']=test_data['abs_diff_latitude']-np.mean(test_data['abs_diff_latitude'])
test_data['abs_diff_latitude']=test_data['abs_diff_latitude']/np.var(test_data['abs_diff_latitude'])


In [ ]:
from sklearn.model_selection import train_test_split
X=train_data.drop(['key','fare_amount'],axis=1)
y=train_data['fare_amount']
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.01,random_state=80)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression(normalize=True)
lr.fit(Xtrain,ytrain)
print(lr.score(Xtest,ytest))

In [ ]:
pred=np.round(lr.predict(test_data.drop('key',axis=1)),2)
submission=pd.DataFrame(data=pred,columns=['fare_amount'])
submission['key']=test_data['key']
submission=submission[['key','fare_amount']]
submission.set_index('key',inplace=True)
submission.to_csv('Submission.csv')